In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
import folium
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Pad naar de bestanden
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'

# Laad de gebruikersprofielen en routes
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing van de routegegevens
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
routes_df['mean_latitude'] = routes_df['latitude'].apply(np.mean)
routes_df['mean_longitude'] = routes_df['longitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['average_location'].tolist(), index=user_profiles_df.index)

# Normaliseer de kenmerken met StandardScaler
scaler = StandardScaler()
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = {}
for index, row in user_profiles_df.iterrows():
    user_id = row['userId']
    gewichten[user_id] = {
        'average_altitude_change': row['weight_std_altitude_change'], 
        'average_distance': row['weight_std_distance'], 
        'average_condition_value': row['weight_std_condition_value'],
        'distance_to_user': 1.0}

# Definieer de functie voor het genereren van nieuwe routes
def generate_new_routes(user_id, user_profiles_df, routes_df, num_segments=5, max_distance=10):
    user_data = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_data['most_common_lat'], user_data['most_common_lon'])

    # Selecteer segmenten die voldoen aan gebruikersvoorkeuren
    # Dit is een gesimplificeerde benadering; een meer geavanceerde methode zou nodig zijn voor een realistische implementatie
    candidate_segments = routes_df[routes_df['distance'] < max_distance]

    # Genereer een nieuwe route door segmenten willekeurig te combineren
    new_route = candidate_segments.sample(n=num_segments)
    return new_route

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_routes(sample_user_id, user_profiles_df, routes_df)

# Visualiseer de nieuwe route
map_obj = folium.Map(location=user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id, ['most_common_lat', 'most_common_lon']].values[0], zoom_start=12)

for _, route_data in new_route.iterrows():
    route_coords = list(zip(route_data['latitude'], route_data['longitude']))
    folium.PolyLine(route_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj
